In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import datetime
import time
import numpy as np
import pandas as pd
import os

# from matplotlib import pyplot as plt
# import plotly.express as px
# %matplotlib inline

In [43]:
def create_year_halfs(years: list):
    '''Обычный ленивый скрипт для создания дат по формату американского времени для гугл трендов'''

    dates, sorted_years = [], sorted(years)
    quarters = ['-06-30', '-12-31']

    # добавляем стартовую точку для нащего периода
    start_date = str(sorted_years[0]) + '-01-01'

    for year in sorted_years:
        quarters_year = [str(year) + date for date in quarters]
        dates.extend(quarters_year)

    dates.insert(0, start_date)  # вставляем start_date в начало списка
    # например, если года 2007-2008, то вернет список ['2007-01-01', '2007-06-30', '2007-12-31', '2008-06-30', '2008-12-31']
    return dates


def download_csv(link: str, sleep_time=5, wait_button_time=120):
    '''Используем билиотеку selenium для автоматического открытия заданной ссылки
       Гугл трендов, робот нажимает на кнопку скачать и закрывает браузер'''

    # открываем драйвер гугла в этой же директории
    driver = webdriver.Chrome(executable_path='./chromedriver')
    time.sleep(5)

    # открываем гугл
    driver.get('https://www.google.ru')
    time.sleep(sleep_time)

    # открываем ссылку гугл трендов по нашему запросу в гугле
    driver.get(link)
    time.sleep(sleep_time)

    # ждем, пока кнопку можно будет нажать
    button = WebDriverWait(driver, wait_button_time).until(EC.element_to_be_clickable(
        (By.CSS_SELECTOR, '.widget-actions-item.export')))

    # скачиваем файл
    button.click()
    time.sleep(5)

    # закрываем браузер
    driver.close()
    pass


def drop_days_columns(df: pd.DataFrame):
    # выдает список столбцов, которые содержат День_ и удаляет их из датафрейма
    columns_to_drop = [
        column for column in df.columns.tolist() if 'День_' in column] # все колонки с датой
    df.rename(columns={columns_to_drop[0]: 'Дата'}, inplace=True) # переименовываем самую первую колонку
    columns_to_drop.pop(0) # удаляем переименованную колонку из списка
    df.drop(labels=columns_to_drop, axis=1, inplace=True) # удаляем оставшиеся колонки с датой из датафрема
    pass


def create_excel_region(ural_regions: dict, benchmark: dict, years: list, queries: list, path='/Users/ska19/Downloads'):
    ''' Принимает на вход словарь ural_regions: ключ название региона (берите полное название из Гугл Трендов) 
        и его код в виде RU-XXX, который будет вставляться в ссылку.
        benchmark - ряд, от которого мы сможем перестроить все ряды.
        Затем список годов years, которые Вам нужны для подкачки.
        И последне queries - слова, по которым вы хотите посмотреть поисковую активность.
        path - ваш путь к папке, куда по умолчанию скачиваются файлы.
        Данный скрипт скачивает ежедневные данные с сайта Гугл тренды, собирает их в датафрейм и затем сохраняет
        в виде эксель файла, используется бенчмарк регионов для последующего перенормирования данных'''

    benchmark_name = list(benchmark.keys())[0]  # название региона бенчмарка
    benchmark_code = list(benchmark.values())[0]  # код региона бенчмарка
    ural_regions_codes = list(ural_regions.values())  # коды всех регионов
    dates = create_year_halfs(years)  # создаем полугодия
    today = pd.Timestamp(datetime.today().date())  # дата на сегодня
    

    for n in range(len(queries)): # идем по каждому запросу
        query = queries[n] # сам запрос
        df_query = pd.DataFrame()  # итоговый датафейм для сохранения в эксельку
        
        # создаем запрос из 5 слов, это максимум для гугл трендов
        for i in range(0, len(ural_regions_codes), 5):
            regions = ural_regions_codes[i:i+5]  # берем 5 регионов
            if benchmark_code not in regions:
                # если бенчмарка нет в списке кодов регионов, то добавим его в список
                extra_region = regions.pop(4) # лишний регионов, вместо него будет вставлен бенчмарк
                # на место лишнего региона вставляем бенчмарк
                ural_regions_codes.insert(ural_regions_codes.index(extra_region), benchmark_code)
                # удаляем лишний регион из начальной позиции
                ural_regions_codes.remove(extra_region)
                # кидаем лишний регион в конец списка
                ural_regions_codes.insert(len(ural_regions_codes)+1, extra_region)
                # вместо лишнего элемента вставляем бенчмарк
                regions.insert(len(regions)+1, benchmark_code)
                # формируем запрос по регионам
                query_regions = ','.join(regions)
                
            print(ural_regions_codes)

            # теперь формируем запрос из слов в query
            # вставляет %20 между словами, если запрос состоит больше, чем из одного слова,
            # затем возвращает столько раз повторенный запрос через запятую, сколько регионов в списке на запрос
            # e.g. : день победы - день%20победы,день%20победы,день%20победы,день%20победы,день%20победы, если регоинов 5
            query_words = ','.join(
                ["%20".join(query.split(' '))] * len(regions))
            query_regions = ','.join(regions)
            df_regions = pd.DataFrame()  # сюда заносим вывод на конкретный запрос по полугодиям

            # идем по полугодиям + 1 день назад из прошлого полугодия
            for j in range(0, len(dates)-1):
                # нам нужно, что бы новое полугодие не выходило за рамки сегодняшней даты
                if today < pd.to_datetime(dates[j], yearfirst=True):
                    print(f"Date {dates[j]} is the future, today is {today}")
                    continue

                # скачиваем файл по ссылке
                link_to_open = f'https://trends.google.ru/trends/explore?date={dates[j]}%20{dates[j+1]}&geo={query_regions}&q={query_words}'
                download_csv(link_to_open)

                # этот кусок тоже можно сделать в отдельную функцию
                # теперь работаем с самим файлом
                # открываем скачанный файл
                df_period = pd.read_csv(path + '/multiTimeline.csv', header=1)
                # переименуем колонку с бенчмарком
                # проверяем, что название региона бенчмарка содержится в колонке
                benchmark_column = [
                    column for column in df_period.columns.tolist() if benchmark_name in column]
                # переименовываем стобик с бенчмарком для join в будущем
                df_period.rename(
                    columns={benchmark_column[0]: benchmark_column[0] + '_' + str(i)}, inplace=True)
                # накладывает фремы друг на друга
                df_regions = pd.concat(
                    [df_regions, df_period], axis=0, ignore_index=True)
                # удаляем файл, чтобы программа могла прочитать следующий при подкачке
                os.remove(path + '/multiTimeline.csv')

            day_column_rename = f'День_{n}_{i}'  # переименовываем дни для теста
            df_regions.rename(
                columns={'День': day_column_rename}, inplace=True)
            # для метода join нужно, что бы индексы совпадали
            df_query.index = df_regions.index
            # добавляем новые запросы к старым по индексу (индексы должны совпадать XD)
            # соединяем таблички друг к другу
            df_query = df_query.join(df_regions)

        # выходим из цикла, работаем с финальной таблицей
        drop_days_columns(df_query)  # удаляем лишние колонки с датой

        # здесь тоже можно написать отдельную функцию
        # заменяем <1 для приведения столбцов к числовому формату
        df_query.replace(to_replace='<1', value=0.5, inplace=True)
        print(df_query.columns)
        df_query['Дата'] = pd.to_datetime(df_query['Дата'])  # делаем формат даты
        # колонки для преобразования к числовому формату
        columns_for_numeric = df_query.drop('Дата', axis=1).columns
        df_query[columns_for_numeric] = df_query[columns_for_numeric].apply(
        pd.to_numeric)  # преобразуем в числовой формат

        # df_query.to_csv('Ural_trends.csv', index=False) # можете сохранять в виде csv, если хотите
        # сохраняем тренды регионов в эксельку
        df_query.to_excel(excel_writer=f'{query}_Ural_trends.xlsx', index=False)
        pass

In [44]:
years = list(range(2022, 2023)) 

ural_regions = {'Свердловская область': 'RU-SVE', 'Курганская область': 'RU-KGN', 'Тюменская область': 'RU-TYU',
                'ХМАО': 'RU-KHM', 'ЯНАО': 'RU-YAN', 'Челябинская область': 'RU-CHE', 'Республика Башкортостан':'RU-BA', 
                'Пермский край': 'RU-PER', 'Оренбургская область': 'RU-ORE'}

benchmark = {'Республика Башкортостан':'RU-BA'}

queries = ['работа', 'вакансии', 'пособия по безработице']

In [45]:
create_excel_region(ural_regions, benchmark, years, queries)

['RU-SVE', 'RU-KGN', 'RU-TYU', 'RU-KHM', 'RU-BA', 'RU-CHE', 'RU-BA', 'RU-PER', 'RU-ORE', 'RU-YAN']


C:\Users\ska19\AppData\Local\Temp/ipykernel_11908/2857612668.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


Date 2022-06-30 is the future, today is 2022-06-11 00:00:00
['RU-SVE', 'RU-KGN', 'RU-TYU', 'RU-KHM', 'RU-BA', 'RU-CHE', 'RU-BA', 'RU-PER', 'RU-ORE', 'RU-YAN']
Date 2022-06-30 is the future, today is 2022-06-11 00:00:00
Index(['Дата', 'работа: (Свердловская область)',
       'работа: (Курганская область)', 'работа: (Тюменская область)',
       'работа: (Ханты-Мансийский автономный округ)',
       'работа: (Республика Башкортостан)_0', 'работа: (Челябинская область)',
       'работа: (Республика Башкортостан)_5', 'работа: (Пермский край)',
       'работа: (Оренбургская область)',
       'работа: (Ямало-Ненецкий автономный округ)'],
      dtype='object')
['RU-SVE', 'RU-KGN', 'RU-TYU', 'RU-KHM', 'RU-BA', 'RU-CHE', 'RU-BA', 'RU-PER', 'RU-ORE', 'RU-YAN']


C:\Users\ska19\AppData\Local\Temp/ipykernel_11908/2857612668.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


Date 2022-06-30 is the future, today is 2022-06-11 00:00:00
['RU-SVE', 'RU-KGN', 'RU-TYU', 'RU-KHM', 'RU-BA', 'RU-CHE', 'RU-BA', 'RU-PER', 'RU-ORE', 'RU-YAN']
Date 2022-06-30 is the future, today is 2022-06-11 00:00:00
Index(['Дата', 'вакансии: (Свердловская область)',
       'вакансии: (Курганская область)', 'вакансии: (Тюменская область)',
       'вакансии: (Ханты-Мансийский автономный округ)',
       'вакансии: (Республика Башкортостан)_0',
       'вакансии: (Челябинская область)',
       'вакансии: (Республика Башкортостан)_5', 'вакансии: (Пермский край)',
       'вакансии: (Оренбургская область)',
       'вакансии: (Ямало-Ненецкий автономный округ)'],
      dtype='object')
['RU-SVE', 'RU-KGN', 'RU-TYU', 'RU-KHM', 'RU-BA', 'RU-CHE', 'RU-BA', 'RU-PER', 'RU-ORE', 'RU-YAN']


C:\Users\ska19\AppData\Local\Temp/ipykernel_11908/2857612668.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='./chromedriver')


Date 2022-06-30 is the future, today is 2022-06-11 00:00:00
['RU-SVE', 'RU-KGN', 'RU-TYU', 'RU-KHM', 'RU-BA', 'RU-CHE', 'RU-BA', 'RU-PER', 'RU-ORE', 'RU-YAN']
Date 2022-06-30 is the future, today is 2022-06-11 00:00:00
Index(['Дата', 'пособия по безработице: (Свердловская область)',
       'пособия по безработице: (Курганская область)',
       'пособия по безработице: (Тюменская область)',
       'пособия по безработице: (Ханты-Мансийский автономный округ)',
       'пособия по безработице: (Республика Башкортостан)_0',
       'пособия по безработице: (Челябинская область)',
       'пособия по безработице: (Республика Башкортостан)_5',
       'пособия по безработице: (Пермский край)',
       'пособия по безработице: (Оренбургская область)',
       'пособия по безработице: (Ямало-Ненецкий автономный округ)'],
      dtype='object')
